In [1]:
from openai import OpenAI
from dotenv import load_dotenv
from openai import OpenAI
from openai.types.beta.assistant_stream_event import ThreadMessageDelta
from openai.types.beta.threads.text_delta_block import TextDeltaBlock 
import os
import pandas as pd

In [2]:
df_scrape = pd.read_pickle('/app/Store/dataframe.pkl')
df_asst = pd.read_csv('/app/Store/assistant_files.csv')
df_asst.rename(columns={'filename': 'file_path'}, inplace=True)

# Merging on the common column 'key'
df_merge = pd.merge(df_scrape, df_asst, on='file_path', how='inner')



In [3]:
result = df_merge[df_merge["file_id"]=='file-DmxSHWuwRB5jv7aWD32V7b']
citation_url = result['url'][0]

In [4]:
load_dotenv('/app/.env')

OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]


def get_asst_by_name(client,name):
    l = client.beta.assistants.list()
    for asst in l:
        asst_dict = asst.to_dict()
        if asst_dict["name"] == name:
            return asst_dict["id"]
        
    return None
        
# Initialise the OpenAI client, and retrieve the assistant
client = OpenAI(api_key=OPENAI_API_KEY)
assistant = client.beta.assistants.retrieve(assistant_id="asst_WvxFY77dyhKaV6ei809wTfye")

In [5]:
user_query = 'Who is Lauren Eichler?'



run = client.beta.threads.create_and_run_poll(
  assistant_id="asst_WvxFY77dyhKaV6ei809wTfye",
  thread={
    "messages": [
      {"role": "user", "content": user_query}
    ]
  },
  tool_choice="required"
)





In [162]:
thread_messages = client.beta.threads.messages.list(run.thread_id)

In [163]:
print(thread_messages)

SyncCursorPage[Message](data=[Message(id='msg_76ZV6PthrsWP0W78SH0h5WWL', assistant_id='asst_WvxFY77dyhKaV6ei809wTfye', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[FileCitationAnnotation(end_index=1357, file_citation=FileCitation(file_id='file-EvUBvU8FUS3KmfpVHmimFs'), start_index=1345, text='【4:0†source】', type='file_citation'), FileCitationAnnotation(end_index=1467, file_citation=FileCitation(file_id='file-APoNCKyyYUxDrJkaducPmM'), start_index=1455, text='【4:2†source】', type='file_citation')], value='You are inquiring about Lauren Eichler, who works at the University of Oregon. \n\nLauren Eichler is the Assistant Director for Advisor Education and Training in the Office of Academic Advising at the University of Oregon. She grew up in Baltimore, Maryland, and after a decade of education and work in New England, she moved to Eugene for graduate school, where she earned her PhD in Philosophy. Her research interests encompass environmental ethics, g

In [164]:
id = thread_messages.data[0].id

In [165]:
# Retrieve the message object
message = client.beta.threads.messages.retrieve(
  thread_id=run.thread_id,
  message_id=id
)


In [166]:
print(message.content[0].text)

Text(annotations=[FileCitationAnnotation(end_index=1357, file_citation=FileCitation(file_id='file-EvUBvU8FUS3KmfpVHmimFs'), start_index=1345, text='【4:0†source】', type='file_citation'), FileCitationAnnotation(end_index=1467, file_citation=FileCitation(file_id='file-APoNCKyyYUxDrJkaducPmM'), start_index=1455, text='【4:2†source】', type='file_citation')], value='You are inquiring about Lauren Eichler, who works at the University of Oregon. \n\nLauren Eichler is the Assistant Director for Advisor Education and Training in the Office of Academic Advising at the University of Oregon. She grew up in Baltimore, Maryland, and after a decade of education and work in New England, she moved to Eugene for graduate school, where she earned her PhD in Philosophy. Her research interests encompass environmental ethics, genocide studies, and Native American philosophies, among other areas.\n\nBefore her current role, Lauren taught philosophy at Old Dominion University, where she also served as the Direc

In [167]:
def get_name_by_file_id(df_merge,file_id):
    result = df_merge[df_merge["file_id"]==file_id]
    citation_url = result['url'].values[0]
    return citation_url

In [168]:

# Extract the message content
message_content = message.content[0].text
annotations = message_content.annotations
citations = []

# Iterate over the annotations and add footnotes
for index, annotation in enumerate(annotations):
    # Replace the text with a footnote
    message_content.value = message_content.value.replace(annotation.text, f' [{index+1}]')
    
    #Get the File ID for the file in this citation
    file_id = annotation.file_citation.file_id

    #Find the URL or source for the file_id that OpenAI has
    citation = get_name_by_file_id(df_merge,file_id)

    # Add the citation to the list
    citations.append(f'[{index+1}] {citation}')

# Add footnotes to the end of the message before displaying to user
message_content.value += '\n' + '\n'.join(citations)

In [169]:
print(message_content.value)

You are inquiring about Lauren Eichler, who works at the University of Oregon. 

Lauren Eichler is the Assistant Director for Advisor Education and Training in the Office of Academic Advising at the University of Oregon. She grew up in Baltimore, Maryland, and after a decade of education and work in New England, she moved to Eugene for graduate school, where she earned her PhD in Philosophy. Her research interests encompass environmental ethics, genocide studies, and Native American philosophies, among other areas.

Before her current role, Lauren taught philosophy at Old Dominion University, where she also served as the Director of Assessment for the Department of Philosophy and Religious Studies. Her passion for mentoring and advising students is evident, and she believes in the importance of liberal arts education as beneficial both to individuals and society.

In her personal life, Lauren enjoys cooking, baking, hiking, board games, and reading science fiction and fantasy. Her comm

In [148]:
import time

formatted_string = message_content.value
delay = 0.01  # delay in seconds

for char in formatted_string:
    print(char, end='', flush=True)
    time.sleep(delay)

You're interested in learning about Lauren Eichler. She is currently the Assistant Director for Advisor Education and Training in the Office of Academic Advising at the University of Oregon.

Lauren Eichler has an extensive academic background, having grown up in Baltimore, MD. She earned her PhD in Philosophy from the University of Oregon after spending a decade in higher education and research. Her areas of interest in philosophy include environmental ethics, genocide studies, and Native American philosophies. Before returning to UO, she taught philosophy at Old Dominion University and served as the Director of Assessment in their Department of Philosophy and Religious Studies. 

In her role at UO, Lauren is deeply committed to mentoring students and guiding them in their academic and professional journeys. She describes advising as a way to engage in meaningful conversations that support students both practically and existentially [1] [2].

If you'd like to know more about her work 